In [73]:
import re
import pandas as pd

In [74]:

def load_data(file_path, skip_first_row=False):
    """
    Loads data from an Excel file into a DataFrame.
    
    Args:
    - file_path (str): The path to the Excel file.
    - skip_first_row (bool): Whether to skip the first row of the file.
    
    Returns:
    - DataFrame: The loaded data.
    """
    skiprows = 1 if skip_first_row else 0
    return pd.read_excel(file_path, skiprows=skiprows)

def assign_temporary_headers(df):
    """
    Assigns temporary headers to the DataFrame for initial examination.
    
    Args:
    - df (DataFrame): The DataFrame to assign temporary headers to.
    
    Returns:
    - DataFrame: The DataFrame with temporary headers assigned.
    """
    temp_headers = [f'Column{i}' for i in range(1, len(df.columns) + 1)]
    df.columns = temp_headers
    return df

In [75]:
def identify_and_assign_correct_headers(df):
    """
    Identifies columns by their content and assigns correct headers
    (pu_quant, pr_codenum, pu_price, and total).
    
    Args:
    - df (DataFrame): The DataFrame with temporary headers.
    
    Returns:
    - DataFrame: The DataFrame with correctly assigned headers.
    """
    # Regex patterns for identifying specific data types
    quantity_pattern = re.compile(r'^\d+$')
    price_pattern = re.compile(r'^\d+\.\d{2}$')
    part_number_pattern = re.compile(r'P\d{2}-\d{3}-\d{3}')

    # Initialize dictionary to hold identified headers
    header_assignments = {'pu_quant': None, 'pr_codenum': None, 'pu_price': None, 'total': None}

    # Iterate over columns to identify their roles
    for col in df.columns:
        # Sample data from column for pattern matching (excluding NaN values)
        sample_data = df[col].dropna().astype(str)

        # Check if column matches specific patterns
        if sample_data.str.match(quantity_pattern).all():
            header_assignments['pu_quant'] = col
        elif sample_data.str.match(part_number_pattern).any():
            header_assignments['pr_codenum'] = col
        elif sample_data.str.match(price_pattern).any():
            # Initial assignment as pu_price; may revise to total later
            if header_assignments['pu_price'] is None:
                header_assignments['pu_price'] = col
            else:
                # Assign the column with higher max value as 'total', the other as 'pu_price'
                current_price_col = header_assignments['pu_price']
                if df[col].max() > df[current_price_col].max():
                    header_assignments['total'] = col
                else:
                    header_assignments['total'] = current_price_col
                    header_assignments['pu_price'] = col

    # Apply identified headers, reorder columns as necessary
    final_headers = {value: key for key, value in header_assignments.items() if value is not None}
    df_renamed = df.rename(columns=final_headers)[list(final_headers.values())]

    return df_renamed

# Note: The implementation assumes that 'total' will always have a higher max value than 'pu_price'.
# Adjustments may be needed depending on the data's specific characteristics.


In [76]:
def clean_data(df):
    """
    Performs cleaning tasks on the DataFrame, such as handling missing values,
    removing duplicates, or any other necessary adjustments.
    
    Args:
    - df (DataFrame): The DataFrame to be cleaned.
    
    Returns:
    - DataFrame: The cleaned DataFrame.
    """
    # Example cleaning steps (customize as needed)

    # Drop rows with any missing values in the columns of interest
    df_cleaned = df.dropna(subset=['pu_quant', 'pr_codenum', 'pu_price', 'total'])

    # Remove duplicate rows (if applicable)
    df_cleaned = df_cleaned.drop_duplicates()

    # Additional cleaning steps could be added here

    return df_cleaned

# This completes the set of functions for a structured approach to processing and cleaning purchase order data.
# Each function handles a specific aspect of the process, allowing for flexibility and iterative refinement.


In [79]:
# Step 1: Load the data
# Replace '/path/to/purchase_orders.xlsx' with the actual file path
file = '3.8_3_table_0test.xlsx'
df_loaded = load_data(f"C:/Users/daniel.pace/Documents/Coding/PO Automation/Azure/Purchase Orders/Output/Archive/{file}", skip_first_row=False)

# Step 2: Assign temporary headers for initial examination
df_temp_headers = assign_temporary_headers(df_loaded)


df_temp_headers.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7
0,50,P11-027-832,4503194D,1/2 G2S White/ Hard rock IPB,EA,27.877,1393.83
1,50,P11-028-218,45214575D,1/2 G2S MERCURY GREY/HARDROCK,EA,34.012,1700.60
2,25,P11-022-166,0,3.2mm G2S,EA,25.680,642.00
3,25,P11-028-218,0,1/2 G2S LUCY PAINT/HARDROCK,EA,34.180,854.50
4,50,P11-028-218,4502369D,1/2 G2S CANOE BIRCH/HARDROCK,EA,34.012,1700.60


In [80]:
# Step 3: Identify and assign correct headers based on data analysis
df_correct_headers = identify_and_assign_correct_headers(df_temp_headers)
df_correct_headers.head()

,pu_quant,pr_codenum,pu_price,total
0,50,P11-027-832,27.877,1393.83
1,50,P11-028-218,34.012,1700.60
2,25,P11-022-166,25.680,642.00
3,25,P11-028-218,34.180,854.50
4,50,P11-028-218,34.012,1700.60


In [71]:

# Step 4: Clean the data by removing unnecessary columns, handling missing values, etc.
df_cleaned = clean_data(df_correct_headers)

# At this point, df_cleaned is ready for further analysis or processing


In [72]:
df_cleaned.head(5)

,pu_quant,pr_codenum,pu_price,total
0,58,P31-041-635,2.73,2507.05
1,7,P31-042-249,9.18,64.26
2,3,P31-042-439|26,10.97,32.91
3,7,P31-042-279,22.89,160.23
4,5,P45-049-036,22.59,112.95
